In [1]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp
import statsmodels as sm
import jupyter as jp
import  sklearn as sk
import yellowbrick as yb




In [4]:

# read the processed_events_final file
df = pd.read_csv('processed_events_final.csv', delimiter='|')





/var/folders/0c/b8f44lfd3p73j8czwx0s68zw0000gn/T/ipykernel_3835/657840752.py:2: DtypeWarning: Columns (6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_events_final.csv', delimiter='|')


In [6]:
# check the first few rows of the dataframe
df.head()

,TIMESTAMP_EVENT,PRICE,PROPOSITION,USER_CLIENT_NUMBER,ARTICLE_CATEGORIE,USER_SESSION_ID,PROMOTION_LABEL,PAGE_NAME,PAGE_SECTION,PAGE_SECTION_POSITION,...,precipitation_coverage_lead_2,precipitation_amount_lead_2,temperature_lead_3,precipitation_coverage_lead_3,precipitation_amount_lead_3,temperature_lead_4,precipitation_coverage_lead_4,precipitation_amount_lead_4,total_spend_on_category_product,total_spend_on_product
0,2022-02-28T14:51:25.349Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.00
1,2022-02-28T14:51:46.443Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.00
2,2022-02-28T14:52:06.207Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.00
3,2022-02-28T14:52:06.211Z,12.5268,349240,386730,"1,091,661,078",933,Actie,Shopping Cart,Recommended Products,1.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.00
4,2022-02-28T14:51:25.319Z,12.5268,349240,386730,"1,091,661,078",933,Actie,Shopping Cart,Recommended Products,1.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9935804,2022-01-28T15:48:08.723Z,35.7192,87264,804960,"1,081,451,429",319213,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,196.88
9935805,2022-01-28T15:46:27.480Z,17.1204,594422,804960,"1,011,031,135",319213,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,734.58
9935806,2022-01-28T15:48:43.837Z,18.2424,8749,804960,"1,011,011,555",319213,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,941.96
9935807,2022-01-28T15:48:53.241Z,1.6500,255956,804960,"1,031,161,491",319213,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,61.05


In [27]:
df.info(), df.shape, df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935809 entries, 0 to 9935808
Data columns (total 42 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   TIMESTAMP_EVENT                  object 
 1   PRICE                            float64
 2   PROPOSITION                      int64  
 3   USER_CLIENT_NUMBER               int64  
 4   ARTICLE_CATEGORIE                object 
 5   USER_SESSION_ID                  int64  
 6   PROMOTION_LABEL                  object 
 7   PAGE_NAME                        object 
 8   PAGE_SECTION                     object 
 9   PAGE_SECTION_POSITION            float64
 10  PROMOTION_PRICE                  float64
 11  PRODUCT_TYPE                     object 
 12  DEVICE_INFO_BRAND                object 
 13  DEVICE_INFO_TYPE                 object 
 14  DEVICE_INFO_BROWSER              object 
 15  USER_SALES_GROUP                 object 
 16  USER_SEGMENT                     object 
 17  USER_SAL

(None,
 (9935809, 42),
 Index(['TIMESTAMP_EVENT', 'PRICE', 'PROPOSITION', 'USER_CLIENT_NUMBER',
        'ARTICLE_CATEGORIE', 'USER_SESSION_ID', 'PROMOTION_LABEL', 'PAGE_NAME',
        'PAGE_SECTION', 'PAGE_SECTION_POSITION', 'PROMOTION_PRICE',
        'PRODUCT_TYPE', 'DEVICE_INFO_BRAND', 'DEVICE_INFO_TYPE',
        'DEVICE_INFO_BROWSER', 'USER_SALES_GROUP', 'USER_SEGMENT',
        'USER_SALES_DISTRICT', 'USER_PROMOTIONS_ALLOWED', 'EVENT', 'DATE',
        'TIME', 'purchases_7_day_after', 'purchases_30_day_after', 'date',
        'temperature', 'precipcover', 'precip', 'temperature_lead_1',
        'precipitation_coverage_lead_1', 'precipitation_amount_lead_1',
        'temperature_lead_2', 'precipitation_coverage_lead_2',
        'precipitation_amount_lead_2', 'temperature_lead_3',
        'precipitation_coverage_lead_3', 'precipitation_amount_lead_3',
        'temperature_lead_4', 'precipitation_coverage_lead_4',
        'precipitation_amount_lead_4', 'total_spend_on_category_product',

### 1. Find the top-selling products (e.g top 5) based on the quantity sold of each of the products

In [40]:
# Function to recommend top-sold products for all clients
def recommend_top_sold_products(dataframe, n=5):
    # Filter out rows where PRICE is 0 to avoid division by zero
    dataframe = dataframe[dataframe['PRICE'] != 0]
    
    # Calculate total money spent on each product
    total_spend_df = dataframe.groupby('PROPOSITION')['total_spend_on_product'].sum().reset_index()
    
    # Merge total spend with price information
    price_df = dataframe[['PROPOSITION', 'PRICE']].drop_duplicates()
    total_spend_df = total_spend_df.merge(price_df, on='PROPOSITION')
    
    # Calculate total quantity sold for each product
    total_spend_df['quantity_sold'] = total_spend_df['total_spend_on_product'] / total_spend_df['PRICE']
    
    # Sort products by total quantity sold in descending order
    total_spend_df = total_spend_df.sort_values(by='quantity_sold', ascending=False)
    
    # Get the list of top N products by quantity sold
    top_products = total_spend_df.head(n)
    
    return top_products


### 2. recommend top-sold products based on the product category that the user buys most

In [41]:
# Function to recommend top-sold products based on the product category that the user buys most
def recommend_top_sold_products_by_category(dataframe, user_client_number, n=5):
    # Filter out rows where PRICE is 0 to avoid division by zero
    dataframe = dataframe[dataframe['PRICE'] != 0]
    
    # Find the product category that the user buys the most
    user_category = dataframe[dataframe['USER_CLIENT_NUMBER'] == user_client_number]['ARTICLE_CATEGORIE'].mode()[0]
    
    # Calculate total money spent on each product within the user's top category
    total_spend_df = dataframe[dataframe['ARTICLE_CATEGORIE'] == user_category].groupby('PROPOSITION')['total_spend_on_product'].sum().reset_index()
    
    # Merge total spend with price information
    price_df = dataframe[['PROPOSITION', 'PRICE']].drop_duplicates()
    total_spend_df = total_spend_df.merge(price_df, on='PROPOSITION')
    
    # Calculate total quantity sold for each product within the user's top category
    total_spend_df['quantity_sold'] = total_spend_df['total_spend_on_product'] / total_spend_df['PRICE']
    
    # Sort products by total quantity sold in descending order
    total_spend_df = total_spend_df.sort_values(by='quantity_sold', ascending=False)
    
    # Get the list of top N products by quantity sold within the user's top category
    top_products = total_spend_df.head(n)
    
    return top_products


In [42]:
# Example of top-sold products recommendation for all clients
top_sold_products_all_clients = recommend_top_sold_products(df, 5)
print("Top Sold Products Recommendation for all clients:")
print(top_sold_products_all_clients)

# Example of category-based top-sold products recommendation for a specific user
user_id = 386730
top_sold_products_user = recommend_top_sold_products_by_category(df, user_id, 5)
print(f"Top Sold Products Recommendation for User {user_id}:")
print(top_sold_products_user)

Top Sold Products Recommendation for all clients:
        PROPOSITION  total_spend_on_product   PRICE  quantity_sold
454409       627917             50669652.56  1.5972   3.172405e+07
585285       802428              5364853.61  0.5940   9.031740e+06
585306       802428              5364853.61  0.6204   8.647411e+06
243491       335036              6161332.30  0.7260   8.486684e+06
243490       335036              6161332.30  0.7392   8.335136e+06
Top Sold Products Recommendation for User 386730:
     PROPOSITION  total_spend_on_product   PRICE  quantity_sold
684       459375             18883333.96  2.8380   6.653747e+06
671       459375             18883333.96  2.8512   6.622943e+06
59         33302             19988629.04  4.2900   4.659354e+06
765       459375             18883333.96  6.9828   2.704264e+06
743       459375             18883333.96  6.9960   2.699162e+06
